# <center>Generating Music Using LSTM Cells</center>

This workbook will implement modified code from [this](https://github.com/corynguyen19/midi-lstm-gan) GitHub repo.

The idea is to read in MIDI files and convert them to arrays of notes. Then an RNN will be trained to predict the next note. Finally, music is generated by feeding a random string of notes to the RNN and having it iteratively predict the next note to form a song one note at a time.

### Things to test tomorrow:

* Add batch normalization - Loss won't go below ~3
* Add an extra layer (RNN and dense) - Meh
* Normalize between -1 to 1 - Didn't do much
* Decrease step size - MUCH better results with step size of 1
* Mess with batch size - Smaller seems better
* Mash-up 2 songs? - Works surprisingly well
* Converges? - No, different inputs should produce different tracks using the same model
* Try transfer learning and training the last few layers on this data?

In [1]:
# Imports
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import os
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.models import load_model
from keras.callbacks import Callback, ModelCheckpoint
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import CuDNNLSTM, LSTM, Bidirectional
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, History

Using TensorFlow backend.


## Loading and Cleaning the Data

First, I will load all the notes from the midi files

In [2]:
def get_notes(path):
    """
        Gets all notes and chords from midi files
    """
    notes = []

    for file in glob.glob(path + "*.mid"):        
        song = []
        midi = converter.parse(file)
        
        print("Parsing %s" % file)
        
        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                song.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                song.append('.'.join(str(n) for n in element.normalOrder))
        notes.append(song)

    return notes

def get_notes_with_key(path, filter_key, mode):
    """
        Gets all notes and chords from midi files where the key matches the string input

        Parameters
        ----------
        path : str
            The path to the file
        filter_key : str
            The string to filter the key on
        mode : int
            The type of key used where:
                0 - key
                1 - major/minor
                else - key and major/minor
    """
    notes = []

    for file in glob.glob(path + "*.mid"):        
        song = []
        midi = converter.parse(file)
        
        # Only use music of the same key
        key = midi.analyze('key')
        if(mode==0):
            key_string = str(key.tonic.name)
        elif(mode==1):
            key_string = str(key.mode)
        else:
            key_string = str(key.tonic.name + key.mode)
            
        if(key_string==filter_key):
            print("Parsing %s" % file)
            notes_to_parse = None

            try: # file has instrument parts
                s2 = instrument.partitionByInstrument(midi)
                notes_to_parse = s2.parts[0].recurse() 
            except: # file has notes in a flat structure
                notes_to_parse = midi.flat.notes

            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    song.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    song.append('.'.join(str(n) for n in element.normalOrder))
            notes.append(song)

    return notes

""" Train a Neural Network to generate music """
# Get notes from midi files
input_dir_choice = 0
input_dir_names = ["Pokemon", "LoZ", "Pokemon GSC"]

input_path = "../" + input_dir_names[input_dir_choice] + " MIDIs/"
# example of each mode: 0 - C, 1 - major, 2 - Cmajor
notes = get_notes_with_key(input_path, "Cmajor", 2)

Parsing ../Pokemon MIDIs\Pokemon - Lavender town.mid
Parsing ../Pokemon MIDIs\Pokemon - Pkmn Elite 4.mid
Parsing ../Pokemon MIDIs\Pokemon - Pokemon Center.mid
Parsing ../Pokemon MIDIs\Pokemon BlackWhite - Surfing.mid
Parsing ../Pokemon MIDIs\Pokemon Channel - Pokepad.mid
Parsing ../Pokemon MIDIs\Pokemon Colosseum - Outskirt Stand.mid
Parsing ../Pokemon MIDIs\Pokemon Colosseum - Relic Forest.mid
Parsing ../Pokemon MIDIs\Pokemon DiamondPearlPlatinum - DialgaPalkia Appear.mid
Parsing ../Pokemon MIDIs\Pokemon DiamondPearlPlatinum - Hearthome City.mid
Parsing ../Pokemon MIDIs\Pokemon DiamondPearlPlatinum - Route 202.mid
Parsing ../Pokemon MIDIs\Pokemon DiamondPearlPlatinum - Route 205.mid
Parsing ../Pokemon MIDIs\Pokemon DiamondPearlPlatinum - Twinleaf Town Day.mid
Parsing ../Pokemon MIDIs\Pokemon GoldSilverCrystal - Ecruteak CityCianwood City.mid
Parsing ../Pokemon MIDIs\Pokemon GoldSilverCrystal - Johto Wild Pokemon Battle.mid
Parsing ../Pokemon MIDIs\Pokemon GoldSilverCrystal - Lavender 

I will now use an algo to determine the key of each song

In [12]:
def print_key(path):
    key_count = dict()
    for file in glob.glob(path + "*.mid"):
        print("Parsing %s" % file)
        
        song = []
        midi = converter.parse(file)
        
        key = midi.analyze('key')
        key_string = key.tonic.name + key.mode
        if (key_string in key_count): 
            key_count[key_string] += 1
        else: 
            key_count[key_string] = 1
        print(key.tonic.name, key.mode)
    return key_count

input_dir_choice = 0
input_dir_names = ["Pokemon", "LoZ", "Pokemon GSC"]

input_path = "../" + input_dir_names[input_dir_choice] + " MIDIs/"
key_count = print_key(input_path)
key_count

Parsing ../Pokemon MIDIs\Pokemon - Farewell, Pikachu!.mid
A minor
Parsing ../Pokemon MIDIs\Pokemon - Lavender town.mid
C major
Parsing ../Pokemon MIDIs\Pokemon - Littleroot Town.mid
F major
Parsing ../Pokemon MIDIs\Pokemon - Lugias Song.mid
E minor
Parsing ../Pokemon MIDIs\Pokemon - Oracion.mid
A minor
Parsing ../Pokemon MIDIs\Pokemon - Pallet Town.mid
E minor
Parsing ../Pokemon MIDIs\Pokemon - Pkmn Elite 4.mid
C major
Parsing ../Pokemon MIDIs\Pokemon - pokecentre theme.mid
D major
Parsing ../Pokemon MIDIs\Pokemon - Pokemon Center Theme.mid
D major
Parsing ../Pokemon MIDIs\Pokemon - Pokemon Center.mid
C major
Parsing ../Pokemon MIDIs\Pokemon - Pokemon Johto - Opening.mid
D major
Parsing ../Pokemon MIDIs\Pokemon - The Ghost at Maiden's Peak.mid
B- minor
Parsing ../Pokemon MIDIs\Pokemon Black & White - Village Bridge.mid
F major
Parsing ../Pokemon MIDIs\Pokemon Black 2White 2 - Cave of Being.mid
F major
Parsing ../Pokemon MIDIs\Pokemon Black 2White 2 - Join Avenue  Greeting.mid
G major
P

KeyboardInterrupt: 

Next, I will find all possible notes and use this to determine how to alter the data to a machine readable format.

In [3]:
possibleNotes = set([item for sublist in notes for item in sublist])

n_vocab = len(possibleNotes)
possibleNotes

{'0',
 '0.1',
 '0.1.3',
 '0.1.4',
 '0.1.5.8',
 '0.1.6',
 '0.2',
 '0.2.4',
 '0.2.4.5',
 '0.2.4.7',
 '0.2.5.8',
 '0.2.6',
 '0.2.7',
 '0.3',
 '0.3.6',
 '0.4',
 '0.4.5.7',
 '0.4.6',
 '0.4.7',
 '0.5',
 '0.5.6',
 '0.6',
 '1',
 '1.2.4',
 '1.3',
 '1.3.6',
 '1.4',
 '1.4.7',
 '1.5',
 '1.5.8',
 '1.6',
 '1.6.7',
 '1.7',
 '10',
 '10.0',
 '10.0.1',
 '10.0.4',
 '10.0.4.5',
 '10.0.5',
 '10.1',
 '10.1.3',
 '10.11',
 '10.11.0',
 '10.11.4',
 '10.2',
 '10.2.3',
 '10.2.3.5',
 '10.2.4',
 '10.2.5',
 '10.3',
 '11',
 '11.0',
 '11.0.2',
 '11.0.4',
 '11.0.4.7',
 '11.1',
 '11.2',
 '11.2.3',
 '11.2.4',
 '11.2.4.7',
 '11.2.5',
 '11.2.6',
 '11.3',
 '11.3.6',
 '11.4',
 '11.4.5',
 '2',
 '2.3',
 '2.3.7',
 '2.4',
 '2.4.7',
 '2.4.9',
 '2.5',
 '2.5.6',
 '2.5.7',
 '2.5.8.11',
 '2.5.9',
 '2.6',
 '2.6.9',
 '2.7',
 '2.8',
 '3',
 '3.4',
 '3.5',
 '3.5.10',
 '3.5.9.11',
 '3.6',
 '3.6.8',
 '3.7',
 '3.7.11',
 '3.8',
 '3.9',
 '4',
 '4.10',
 '4.5',
 '4.5.9',
 '4.5.9.0',
 '4.6',
 '4.6.9',
 '4.7',
 '4.7.10',
 '4.7.11',
 '4.7.9',
 '4.7

Now I will prepare the sequences of notes by looking at each song individually. I will first grab an arrays of size **sequence_length** with a stride of **step_size** from each song. Then I will map the chords to integers so the model can learn from that and normalize the input between 0-1.

In [4]:
def prepare_sequences(notes, allNotes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100
    step_size = 1

    # get all pitch names
    pitchnames = sorted(allNotes)

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for song in notes:
        for i in range(0, len(song) - sequence_length, step_size):
            sequence_in = song[i:i + sequence_length]
            sequence_out = song[i + sequence_length]
            network_input.append([note_to_int[char] for char in sequence_in])
            network_output.append(note_to_int[sequence_out])

    # reshape the input into a format compatible with LSTM layers
    n_patterns = len(network_input)
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # normalize input between 0 and 1
    network_input = network_input / float(n_vocab)
    
    # normalize input between -1 and 1
#     network_input = (network_input / float(n_vocab))*2 - 1

    # Make one-hot-encoding
    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

network_input, network_output = prepare_sequences(notes, possibleNotes, n_vocab)
len(network_input)

13637

## Constructing the model

I will now construct the model using CuDNNLSTM cells because they are significantly faster than regular LSTM cells due to being optimized for CuDA. I will have two CuDNNLSTM layers, followed by two dense layers and a final softmax activation layer to output the most probable result.

Hyperparameters:
* Optimizer - ADAM because it is considered one of the best
* Loss - categorical_crossentropy because it penalizes wrong predictions of multi-class problems best
* Epochs - More epochs are generally better as long as they don't overfit. I track loss over time and have checkpoints every 5 epochs so this will not be a problem
* Batch Size - This determines how many instances should be considered in each batch. Realistically, each different song would interfere with the other so I will reduce this.
    * Smaller Batch Size seems to add more variation

In [42]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(CuDNNLSTM(512,input_shape=(network_input.shape[1], network_input.shape[2]),return_sequences=True))
    model.add(Dropout(0.3))
    model.add(Bidirectional(CuDNNLSTM(512, return_sequences=True)))
    model.add(Dropout(0.3))
    model.add(Bidirectional(CuDNNLSTM(512)))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

# Set up the model
model = create_network(network_input, n_vocab)
history = History()

# Save on each epoch (because training isn't cheap!!!) and can use this to generate music for each checkpoint
outputDest = '../output/LSTM_' + input_dir_names[input_dir_choice] + '_' + str(int(time.time())) + '/'
if not os.path.exists(outputDest):
    os.makedirs(outputDest)

cp_callback = ModelCheckpoint(filepath=outputDest + "LSTMmodel_weights_{epoch:02d}.hdf5",
                              save_weights_only=True,
                              verbose=1,
                              period=10)

# Set parameters
n_epochs = 100
batch_size = 10
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_17 (CuDNNLSTM)    (None, 100, 512)          1054720   
_________________________________________________________________
dropout_17 (Dropout)         (None, 100, 512)          0         
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 100, 1024)         4202496   
_________________________________________________________________
dropout_18 (Dropout)         (None, 100, 1024)         0         
_________________________________________________________________
bidirectional_12 (Bidirectio (None, 1024)              6299648   
_________________________________________________________________
dense_11 (Dense)             (None, 256)               262400    
_________________________________________________________________
dropout_19 (Dropout)         (None, 256)              

## Training the Model

I will save the final model, but keep checkpoints along the way to avoid overfitting and also use these to generate different midis.

In [43]:
model.fit(network_input, network_output, callbacks=[history, cp_callback], epochs=n_epochs, batch_size=batch_size)
model.save(outputDest + 'LSTMmodel_final.h5')

Epoch 1/100
169/169 [==============================] - 6s 33ms/step - loss: 3.2524
Epoch 2/100
169/169 [==============================] - 1s 8ms/step - loss: 2.9582
Epoch 3/100
169/169 [==============================] - 1s 8ms/step - loss: 2.8854
Epoch 4/100
169/169 [==============================] - 1s 8ms/step - loss: 2.8786
Epoch 5/100
169/169 [==============================] - 1s 8ms/step - loss: 2.8976
Epoch 6/100
169/169 [==============================] - 1s 8ms/step - loss: 2.8661
Epoch 7/100
169/169 [==============================] - 1s 8ms/step - loss: 2.8366
Epoch 8/100
169/169 [==============================] - 1s 8ms/step - loss: 2.8232
Epoch 9/100
169/169 [==============================] - 1s 8ms/step - loss: 2.8119
Epoch 10/100
169/169 [==============================] - 1s 8ms/step - loss: 2.7749

Epoch 00010: saving model to ../output/LSTM_Pokemon_1570420091/LSTMmodel_weights_10.hdf5
Epoch 11/100
169/169 [==============================] - 1s 8ms/step - loss: 2.7302
Epoch

Epoch 91/100
169/169 [==============================] - 1s 8ms/step - loss: 0.0018
Epoch 92/100
169/169 [==============================] - 1s 8ms/step - loss: 0.0015
Epoch 93/100
169/169 [==============================] - 1s 8ms/step - loss: 0.0015
Epoch 94/100
169/169 [==============================] - 1s 8ms/step - loss: 0.0012
Epoch 95/100
169/169 [==============================] - 1s 8ms/step - loss: 0.0018
Epoch 96/100
169/169 [==============================] - 1s 8ms/step - loss: 0.0011
Epoch 97/100
169/169 [==============================] - 1s 8ms/step - loss: 0.0015
Epoch 98/100
169/169 [==============================] - 1s 8ms/step - loss: 0.0012
Epoch 99/100
169/169 [==============================] - 1s 8ms/step - loss: 0.0014
Epoch 100/100
169/169 [==============================] - 1s 8ms/step - loss: 0.0012

Epoch 00100: saving model to ../output/LSTM_Pokemon_1570420091/LSTMmodel_weights_100.hdf5


In [44]:
# Plot the model losses
pd.DataFrame(history.history).plot()
plt.savefig(outputDest + 'LSTM_Loss_per_Epoch.png', transparent=True)
plt.close()

## Generating Music

I will now use the model to generate music by feeding it a random string of notes and have it predict the next one, then have it predict the one after that until a full song has been generated.

In [45]:
def generate_notes(model, notes, network_input, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    pitchnames = sorted(notes)
    
    start = np.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)
#         prediction_input = (prediction_input / float(n_vocab))*2 - 1

        prediction = model.predict(prediction_input, verbose=0)

        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)
        
        pattern = np.append(pattern,index)
        pattern = pattern[1:len(pattern)]
    
    print(prediction_output)

    return prediction_output

# prediction_output = generate_notes(model, possibleNotes, network_input, n_vocab)

Next, I will create a midi using these notes and save to a file

In [46]:
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))
    
# create_midi(prediction_output, outputDest + 'LSTM_output_final')

Alternatively, I can run this script to convert all of the models into midi files and select my favourite from a much larger album.

In [47]:
# Have each model make a song
count = 0
for model_path in glob.glob(outputDest + "*.hdf5"):
    print("Composing from %s" % model_path)
    model.load_weights(model_path)
    prediction_output = generate_notes(model, possibleNotes, network_input, n_vocab)
    create_midi(prediction_output, outputDest + 'LSTM_output_' + str(count))
    print(outputDest + 'LSTM_output_' + str(count))
    count += 1

Composing from ../output/LSTM_Pokemon_1570420091\LSTMmodel_weights_10.hdf5
['D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'D3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 'E3', 

../output/LSTM_Pokemon_1570420091/LSTM_output_2
Composing from ../output/LSTM_Pokemon_1570420091\LSTMmodel_weights_30.hdf5
['2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '

../output/LSTM_Pokemon_1570420091/LSTM_output_5
Composing from ../output/LSTM_Pokemon_1570420091\LSTMmodel_weights_60.hdf5
['2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '

../output/LSTM_Pokemon_1570420091/LSTM_output_8
Composing from ../output/LSTM_Pokemon_1570420091\LSTMmodel_weights_90.hdf5
['2.7', '2.7', '2.7', '9.0', '0.5', '9.0', '0.5', '2.7', '2.7', '2.7', '2.7', '2.7', '9.0', '0.5', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '9.0', '0.5', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '0.5', '2.7', '2.7', '2.7', '2.7', '2.7', '0.5', '0.5', '2.7', '2.7', '2.7', '2.7', '9.0', '0.5', '0.5', '2.7', '2.7', '2.7', '2.7', '2.7', '9.0', '0.5', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '2.7', '0.5', '0.5', '0.5', '2.7', '2.7', '2.7', '2.7', '0.5', '9.0', '0.5', '2.7', '2.7', '2.7', '0.5', '0.5', '2.7', '2.7', '2.7', '2.7', '2.7', '0.5', '0.5', '0.5', '2.7', '2.7', '2.7', '2.7', '9.0', '0.5', '